In [187]:
import h5py
import numpy as np
import capnp
import sys
sys.path.append('/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt')

import CapnpGame_capnp


In [188]:
raw_data = h5py.File('/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/example.h5','r')


In [232]:
val_prob = 0.05 

# create file for dataset
dataset = h5py.File('/Users/valentinwolf/Documents/Studium/Machine_Learning/SoftwareProjekt/train_val.h5', 'w', libver='latest')
samples = 25000

dataset.swmr_mode = True
train_x = dataset.create_dataset("train_x", shape=(int(samples*(1-val_prob)),12,9,9), dtype='int8')
train_y = dataset.create_dataset("train_y", shape=(int(samples*(1-val_prob)),83))
val_x = dataset.create_dataset("val_x", shape=(int(samples*val_prob),12,9,9), dtype='int8')
val_y = dataset.create_dataset("val_y", shape=(int(samples*val_prob),83))

val_x.attrs.create("next_i_to_overwrite",0)
train_x.attrs.create("next_i_to_overwrite",0)

In [201]:
val_x.shape[0]

25

In [240]:
# probability of a specific game is chosen for the validation set
game_ids_dict = {}
boardsize = 9

for file in raw_data['game_record']:
    game_msg = file.tostring()
    g = CapnpGame_capnp.Game.from_bytes(game_msg)
    
    if np.random.sample(1)[0] > val_prob:
        dest_x, dest_y = train_x, train_y
    else:
        dest_x, dest_y = val_x, val_y
        
    for i in range(len(g.stateprobs)):
        offset = dest_x.attrs["next_i_to_overwrite"]
        dest_x[offset] = np.array(g.stateprobs[i].state).reshape((4 + 8 , 9, 9))
        
        probs = np.array(g.stateprobs[i].probs)
        winner = np.asarray([int(g.result)])

        y = np.concatenate((probs.flatten(), winner), axis=0)

        dest_y[offset] = y.reshape((1,83))
        
        dest_x.attrs.modify("next_i_to_overwrite",(offset+1)%dest_x.shape[0])
        

In [242]:
train_x.attrs["next_i_to_overwrite"]

1522